In [3]:
from onnxruntime_extensions.onnxprocess import trace_for_onnx
from onnxruntime_extensions.onnxprocess import torch_wrapper as torch  # overload torch API if it is needed
import numpy as np

ModuleNotFoundError: No module named 'torch.types'

In [9]:
from pathlib import Path
img_path = Path('/home/mazheng/waste-detection/Yet-Another-EfficientDet-Pytorch/onnx_inference/examples/fig')

In [16]:
import torch
import numpy as np

In [17]:
def func():
    return np.array([1]),np.array([2]),np.array([3])

In [20]:
a,b,c = [torch.from_numpy(i) for i in func()]

In [21]:
a,b,c

(tensor([1]), tensor([2]), tensor([3]))

In [ ]:
def postprocess(
    x,
    anchors,
    regression,
    classification,
    regressBoxes,
    clipBoxes,
    threshold,
    iou_threshold,
):
    transformed_anchors = regressBoxes(anchors, regression)
    transformed_anchors = clipBoxes(transformed_anchors, x)
    scores = torch.max(classification, dim=2, keepdim=True)[0]
    # print(scores)
    # print(scores > threshold)
    scores_over_thresh = (scores > threshold)[:, :, 0]
    out = []
    for i in range(x.shape[0]):
        if scores_over_thresh[i].sum() == 0:
            # print(scores_over_thresh[i].sum())
            out.append(
                {
                    "rois": np.array(()),
                    "class_ids": np.array(()),
                    "scores": np.array(()),
                }
            )
            continue

        classification_per = classification[i, scores_over_thresh[i, :], ...].permute(
            1, 0
        )
        transformed_anchors_per = transformed_anchors[i, scores_over_thresh[i, :], ...]
        scores_per = scores[i, scores_over_thresh[i, :], ...]
        scores_, classes_ = classification_per.max(dim=0)
        anchors_nms_idx = batched_nms(
            transformed_anchors_per,
            scores_per[:, 0],
            classes_,
            iou_threshold=iou_threshold,
        )

        if anchors_nms_idx.shape[0] != 0:
            classes_ = classes_[anchors_nms_idx]
            scores_ = scores_[anchors_nms_idx]
            boxes_ = transformed_anchors_per[anchors_nms_idx, :]

            out.append(
                {
                    "rois": boxes_.cpu().numpy(),
                    "class_ids": classes_.cpu().numpy(),
                    "scores": scores_.cpu().numpy(),
                }
            )
        else:
            out.append(
                {
                    "rois": np.array(()),
                    "class_ids": np.array(()),
                    "scores": np.array(()),
                }
            )

    return out

